In [37]:
from skimage import feature
from matplotlib import pyplot as plt

import numpy as np
from sklearn.svm import LinearSVC  
from imutils import paths
import argparse
import cv2
from keras.datasets import cifar10


In [38]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [97]:
numPoints = 96
radius = 1
def describe(image, eps=1e-7):
# compute the Local Binary Pattern representation
# of the image, and then use the LBP representation
# to build the histogram of patterns
    lbp = feature.local_binary_pattern(image,numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, numPoints + 3), range=(0, numPoints + 2)) 
# # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + eps)
    # return the histogram of Local Binary Patterns
    return hist

In [98]:
data = []
labels = []
for i in range(0,x_train.shape[0]):
    img = x_train[i]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist = describe(gray)
    labels.append(y_train[i])
    data.append(hist)
#     print(hist.shape)
#     break
# np.save('train_data.npy',data)
# np.save('train_label.npy',labels)    

In [99]:
# train a Linear SVM on the data
# data = np.load('train_data.npy')
# labels = np.load('train_label.npy')   
svclassifier = LinearSVC()  
h = svclassifier.fit(data, labels)  

/home/anirudh/detect/lib/python3.5/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [100]:
acc = [] 
# loop over the testing images
for i in range(0,x_test.shape[0]):
    img = x_test[i]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist = describe(gray)
    prediction = h.predict(hist.reshape(1, -1))
    acc.append(prediction == y_test[i])

In [101]:
np.mean(acc)


0.2975

In [102]:
np.sum(acc*1)

2975

In [103]:
x_test.shape

(10000, 32, 32, 3)